# **Running agents**

You can run agents via the Runner class. You have 3 options:

- `Runner.run()`, which runs async and returns a `RunResult`.
- `Runner.run_sync()`, which is a sync method and just runs `.run()` under the hood.
- `Runner.run_streamed()`, which runs async and returns a `RunResultStreaming`. It calls the LLM in streaming mode, and streams those events to you as they are received.


In [2]:
# load env variables for using custom llm 
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY environment variable is not set. Please set it to your Gemini API key.")

In [3]:

# load config from openai_agents.config import OpenAIChatCompletionsModel, AsyncOpenAI
from agents import OpenAIChatCompletionsModel, AsyncOpenAI

async def create_gemini_config():
    """Create Gemini configuration"""
    if not gemini_api_key:
        return None, None
    
    provider = AsyncOpenAI(
        api_key=gemini_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    
    model = OpenAIChatCompletionsModel(
        model="gemini-2.0-flash-exp",
        openai_client=provider
    )
    
    print(f"Using Gemini model: *** {model.model} ***")
    
    return provider, model

In [4]:
# now create run config
from agents import RunConfig

provider, model = await create_gemini_config()
run_config = RunConfig(
    model=model,
    model_provider=provider,
    tracing_disabled=True
)

Using Gemini model: *** gemini-2.0-flash-exp ***


In [25]:
from agents import Agent, Runner

In [26]:
agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = await Runner.run(agent, "Write a haiku about recursion in programming.", run_config=run_config)

print(result.final_output)

Call the same again,
A loop within the code,
Ending is the key.



### Markdown documentation about Runner.run() default behavior

#### Runner.run() Default Status and Behavior

#### Execution Model
- **Asynchronous**: `Runner.run()` is an async function that must be awaited
- **Returns**: `RunResult` object containing the agent's response
- **Non-blocking**: Allows other async operations to run concurrently

### Usage Pattern

In [27]:
# Example #01
# Runner.run(), which runs async and returns a RunResult.
async def main(prompt : str):
    agent = Agent(name="Assistant", instructions="You are a helpful assistant")

    result = await Runner.run(agent, prompt, run_config=run_config)
    return result.final_output
    # Code within the code,
    # Functions calling themselves,
    # Infinite loop's dance
    
result = await main("Write a haiku about recursion in programming.")
print(result)



Call the same again,
Smaller problem takes the lead,
Base case sets us free. 



In [28]:
await main("Write a poem of 10 words in rainy day.")


'Gray skies weep, the world is washed, a gentle, soothing sleep.\n'

In [ ]:
from g_config import create_openai_config

ai_provider, ai_model = create_openai_config()

Using OpenAI model: *** gpt-4o-mini ***


In [69]:
# openai configrations
opainai_run_config = RunConfig(
    model=ai_model,
    model_provider=ai_provider,
    tracing_disabled=True
)

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

In [ ]:
# stream agent with gpt response
stream_agent = Agent(
    name="StreamAgent",
    instructions="You are a streaming assistant that provides responses in real-time."
)

result = Runner.run_streamed(stream_agent, input="Please tell me 5 jokes.", run_config=opainai_run_config)

async for event in result.stream_events():
    if event.type == "raw_response_event":
        # Handle different types of response events
        try:
            if hasattr(event.data, 'delta') and event.data.delta:
                # Handle delta events (streaming content)
                if hasattr(event.data.delta, 'content'):
                    print(event.data.delta.content, end="", flush=True)
                elif hasattr(event.data.delta, 'text'):
                    print(event.data.delta.text, end="", flush=True)
            elif hasattr(event.data, 'content'):
                # Handle direct content
                print(event.data.content, end="", flush=True)
        except Exception as e:
            # Skip problematic events and continue streaming
            continue